In [4]:
import torch
from transformers import Trainer, AutoTokenizer, TrainingArguments
from model.model import HatefulMemesData, HateMemeClassifier, compute_metrics
from safetensors.torch import load_file
import os

import matplotlib.pyplot as plt
import json
import cv2
import numpy as np
from copy import deepcopy

from transformers import ViTFeatureExtractor, ViTModel
from PIL import Image
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, VisualBertModel, VisualBertConfig
from torch.nn import CrossEntropyLoss
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from sklearn.metrics import roc_auc_score
from datasets import load_metric

In [8]:
class HateMemeClassifier(torch.nn.Module):
    def __init__(self,fusion_method, visual_embedder='vit',wandb_run=None):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        fusion_method: 'concatenate' or 'weight_ensemble' or 'linear_weight_ensemble' or 'visualbert'
        visual_embedder: 'vit'
        """
        super(HateMemeClassifier, self).__init__()
        self.fusion_method = fusion_method # 'concatenate' or 'weight_ensemble' or 'linear_weight_ensemble' or 'visualbert'
        self.wandb_run=wandb_run

        configuration = VisualBertConfig.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre',
                                                hidden_dropout_prob=0.1, attention_probs_dropout_prob=0.1)
        self.visualbert = VisualBertModel.from_pretrained('uclanlp/visualbert-nlvr2-coco-pre', config=configuration)
        if self.fusion_method != 'visualbert':
            self.bertmodel = BertModel.from_pretrained('bert-base-uncased')

        if visual_embedder=='vit':
            self.embed_cls = nn.Linear(768, 1024)
        self.num_labels = 2
        self.dropout = nn.Dropout(0.3)
    
        if self.fusion_method=='weight_ensemble':
            self.alpha = nn.Parameter(torch.tensor(0.5))  # Initial value of alpha
            
        if self.fusion_method=='linear_weight_ensemble':
            self.alpha = nn.Parameter(torch.tensor(0.5))  # Initial value of alpha
            self.cls_visual = nn.Sequential(
                nn.Linear(768, 768),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.BatchNorm1d(768),
            )
            self.cls_text = nn.Sequential(
                nn.Linear(768, 768),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.BatchNorm1d(768),
            )

        self.cls=nn.Linear(768, self.num_labels)

        # Calculate the weights for the loss function and weight balanced loss
        nSamples = [5450,3050]
        normedWeights = [1 - (x / sum(nSamples)) for x in nSamples]
        self.loss_fct = CrossEntropyLoss(weight=torch.FloatTensor(normedWeights))

        # self.loss_fct = CrossEntropyLoss()
        
    
    def forward(self, input_ids, attention_mask, token_type_ids, visual_embeds, visual_attention_mask,
                visual_token_type_ids, labels,caption_input_ids, caption_attention_mask, caption_token_type_ids):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        visual_embeds_cls = self.embed_cls(visual_embeds)
        
        outputs = self.visualbert(
                input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                visual_embeds=visual_embeds_cls,
                visual_attention_mask=visual_attention_mask,
                visual_token_type_ids=visual_token_type_ids,
            )
        
        visualbert_embedding = outputs[1] # output is a context vector of 768 dimensions

        if self.fusion_method != "visualbert":
        
            caption_outputs = self.bertmodel(caption_input_ids, attention_mask=caption_attention_mask, token_type_ids=caption_token_type_ids)
                    
            # Get the embeddings of the [CLS] token
            caption_embeddings = caption_outputs.last_hidden_state[:,0,:] # output is a context vector of 768 dimensions
        
        if self.fusion_method=='weight_ensemble':
            # funsion model: weight ensenble of the two embeddings: alpha*visualbert_embedding + (1-alpha)*caption_embeddings 
            fused_embedding = self.alpha * self.dropout(visualbert_embedding) + (1-self.alpha) * self.dropout(caption_embeddings)
            # self.wandb_run.log({"alpha": self.alpha.data.cpu().numpy()},commit=False)
        
            logits = self.cls(fused_embedding)
        if self.fusion_method=='linear_weight_ensemble':
            # funsion model: weight ensenble of the two embeddings: alpha*visualbert_embedding + (1-alpha)*caption_embeddings 
            fused_embedding = self.alpha * self.cls_visual(visualbert_embedding) + (1-self.alpha) * self.cls_text(caption_embeddings)
            self.wandb_run.log({"alpha": self.alpha.data.cpu().numpy()},commit=False)
        
            logits = self.cls(fused_embedding)
        
        if self.fusion_method=='visualbert':
            logits = self.cls(self.dropout(visualbert_embedding))
            
        if self.fusion_method=='concatenate':
            # funsion model: concatenate the two embeddings
            fused_embedding = torch.cat((visualbert_embedding, caption_embeddings), dim=1)
            logits = self.cls(fused_embedding)
        
        
        reshaped_logits = logits.view(-1, self.num_labels)
        loss = self.loss_fct(reshaped_logits, labels.view(-1))
      
        return loss, reshaped_logits

In [9]:


# Prepare directories
data_folder_path = "/scratch/izar/cchang/EE559/data/hateful_memes" 
checkpoint_path = "/scratch/izar/cchang/EE559/model-checkpoint/hatefulmemcladdifier_weight_ensemble_vit_smooth-totem-17_202405230523/checkpoint-3000"
query = 'query_8' 

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Define paths for evaluation data
validation_data_path = os.path.join(data_folder_path, 'query236/dev_seen_.jsonl')
img_inpainted_dir = os.path.join(data_folder_path, 'img')
visual_embed_model = 'vit'  
fusion_method = 'weight_ensemble'
seq_len = 50

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HateMemeClassifier(fusion_method=fusion_method, visual_embedder=visual_embed_model)
model = model.to(device)

# Load the model weights from the safetensors file
state_dict = load_file(os.path.join(checkpoint_path, "model.safetensors"))
model.load_state_dict(state_dict)

# Prepare evaluation dataset
eval_dataset = HatefulMemesData(
    validation_data_path, 
    img_inpainted_dir, 
    tokenizer, 
    sequence_length=seq_len, 
    query=query, 
    visual_embed_model=visual_embed_model, 
    device=device
)

# Define training arguments (only evaluation relevant parameters)
args = TrainingArguments(
    output_dir="./results",  
    per_device_eval_batch_size=24
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Evaluate the model
metrics = trainer.evaluate()
print(metrics)


/home/cchang/miniconda3/envs/visualbert/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


wandb: Currently logged in as: changchuntzu (jessica-chang). Use `wandb login --relogin` to force relogin


{'eval_loss': 2.3082611560821533, 'eval_accuracy': 0.644, 'eval_auroc': 0.6419804451841065, 'eval_f1': 0.5679611650485437, 'eval_precision': 0.7090909090909091, 'eval_recall': 0.47368421052631576, 'eval_runtime': 38.7032, 'eval_samples_per_second': 12.919, 'eval_steps_per_second': 0.543}
